In [70]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador, CVM
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
import numpy as np

In [83]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1()
cvm = CVM()

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = fdt.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Data_ini_boletas
data_ini = pd.datetime(2024,1,1)

In [166]:
#Retorna a carteira dos fundos que estão encerrando

fd_encerramento = pd.read_excel('C:/Temp/Foundation/Encerramento/encerramento_fundos.xlsx')
po_cadastro = cadastro= posicao.po_cadastro()[['NomeContaCRM','Officer','Controller','CNPJ','Segmento']]
po_cadastro['CNPJ'] = po_cadastro['CNPJ'].astype('str')
po_posicao = posicao.posicao_fundos_all(hoje)


In [167]:
ativos_desconsiderar = ['COMP','CAIXA','Ajustes']
ativos_simples = ['NTN-B','NTN-C','NTN-F', 'LTN','LFT','AÇOES','FUT','BOLSA']

fundos = list(fd_encerramento['CNPJ'])
fundos = [x.replace('-','').replace('.','').replace('/','')   for x in fundos ]

cadastro = cadastro.loc[cadastro['CNPJ'].isin(fundos)][['NomeContaCRM']]
cadastro = list(cadastro['NomeContaCRM'])

pos_fd_encerramento = po_posicao.loc[~po_posicao['TipoProduto'].isin(ativos_desconsiderar)]
pos_fd_encerramento = po_posicao.loc[po_posicao['NomeContaCRM'].isin(cadastro)]
pos_fd_encerramento['GuidContaCRM'] = pos_fd_encerramento['GuidContaCRM'].str.lower()
guids = list(pos_fd_encerramento['GuidContaCRM'])
pos_fd_encerramento = pos_fd_encerramento[['GuidContaCRM','NomeContaCRM','DataArquivo','TipoProduto','Classe','GuidProduto','NomeProduto','FormaCondominio','QuantidadeFinal','QtdeBloq','FinanceiroFinal','ResgPedido','ResgCot','ResgFin']]
pos_fd_encerramento['Disponivel_Secundario'] = pos_fd_encerramento.apply(lambda row : row['QtdeBloq'] if row['TipoProduto'] not in ativos_simples else np.nan,axis=1 )
pos_fd_encerramento = pos_fd_encerramento[['DataArquivo','GuidContaCRM','NomeContaCRM','TipoProduto','Classe','GuidProduto','NomeProduto','FormaCondominio','QuantidadeFinal','QtdeBloq','Disponivel_Secundario','FinanceiroFinal','ResgPedido','ResgCot','ResgFin']].rename(columns={'QtdeBloq':'QtdeBloq_Margem'})


In [168]:
boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=hoje, exportado=True, deletado=False).reset_index()
boletas = boletas[(boletas['StatusCRM']!='Cancelada')&(boletas['StatusCRM']!='Devolvida')]
boletas = boletas[boletas['DataCot']>hoje]
boletas['ContaCRMGuid'] = boletas['ContaCRMGuid'].str.lower()
guids_v2 = [x.replace('{','').replace('}','') for x in boletas['ContaCRMGuid']]
boletas['ContaCRMGuid'] = guids_v2
boletas = boletas.loc[boletas['ContaCRMGuid'].isin(guids)]
boletas = boletas[boletas['TipoMov']=='V']
boletas['Movimentacao_Solicitada'] = 'Sim'
boletas = boletas[['ContaCRMGuid','AtivoGuid','Movimentacao_Solicitada','TipoMov','ResgTot','DataMov','DataCot','DataFin','StatusCRM']]

#INcluir nas informações de posição
verificacao = pd.merge(left = pos_fd_encerramento,right= boletas,left_on=['GuidContaCRM','GuidProduto'],right_on=['ContaCRMGuid','AtivoGuid'], how='outer')

verificacao = verificacao[verificacao['DataArquivo'].notnull()].sort_values(by=['NomeContaCRM'])

In [171]:
verificacao[['DataArquivo','NomeProduto','Classe','QuantidadeFinal','QtdeBloq_Margem','Disponivel_Secundario','FormaCondominio','FinanceiroFinal','Movimentacao_Solicitada','ResgTot','DataMov','DataCot','DataFin','StatusCRM']]

,DataArquivo,NomeProduto,Classe,QuantidadeFinal,QtdeBloq_Margem,Disponivel_Secundario,FormaCondominio,FinanceiroFinal,Movimentacao_Solicitada,ResgTot,DataMov,DataCot,DataFin,StatusCRM
429,2024-06-26,"SB FIDC Multissetorial - 5S1E SR CDI+4,50% 16/...",R Fixa Pós,62.000000,NaN,NaN,Fechado,66398.40,NaN,NaN,NaT,NaT,NaT,NaN
1540,2024-06-26,LF Banco Daycoval SA 100% CDI 1% 06/Dez/24,R Fixa Pós,1.000000,NaN,NaN,t:11,60963.96,NaN,NaN,NaT,NaT,NaT,NaN
2132,2024-06-26,JBFO RV Tático FIA - BDR Nível I (BRL),R Variável,320191.189354,NaN,NaN,Aberto,466333.45,NaN,NaN,NaT,NaT,NaT,NaN
1964,2024-06-26,"Axis Renovaveis FIDC - 1S SR IPCA+7,45% 29/Jan...",R Fixa Infl,161.999993,NaN,NaN,Fechado,157032.91,NaN,NaN,NaT,NaT,NaT,NaN
2774,2024-06-26,Ajuste - AJUSTE (BRL),Ajuste,1.000000,NaN,NaN,Indeterminado,-369.09,NaN,NaN,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,2024-06-26,Companhia de Concessões Rodoviárias - CCRO3,R Variável,-1500.000000,NaN,NaN,t:9,-17880.00,NaN,NaN,NaT,NaT,NaT,NaN
486,2024-06-26,JBFO Corporativo 60 FIM CrPr (BRL),R Fixa Pós,242837.224654,NaN,NaN,Aberto,584838.32,Sim,True,2024-06-25,2024-08-26,2024-08-27,Movimentação Solicitada
1312,2024-06-26,JBFO Genoa Capital Radar FIC FIM CrPr (BRL),Alternativos,137546.208649,NaN,NaN,Aberto,220013.92,NaN,NaN,NaT,NaT,NaT,NaN
2817,2024-06-26,JBFO Augme FIC FIM CrPr (BRL),R Fixa Pós,380461.998048,NaN,NaN,Aberto,580601.22,Sim,True,2024-05-31,2024-08-28,2024-08-29,Movimentação Solicitada
